In [ ]:
!pip install openai groq gradio PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json
import os
import requests
from PyPDF2 import PdfReader

In [ ]:
from google.colab import userdata

In [ ]:
groq_api_key=userdata.get("GROQ_API_KEY")
groq_base_url="https://api.groq.com/openai/v1"

In [ ]:
groq=OpenAI(api_key=groq_api_key, base_url=groq_base_url)

In [ ]:
pushover_user=userdata.get("PUSHOVER_USER")
pushover_token=userdata.get("PUSHOVER_TOKEN")
pushover_url="https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
  print(f"push: {message}")
  payload={"user":pushover_user,"token":pushover_token, "message":message}
  requests.post(pushover_url,data=payload)

In [ ]:
push("Hi!")

push: Hi!


# tools

In [ ]:
def record_user_details(email, name="Name not provied",notes="Notes not provided"):
  push(f"Recording {name} with email {email} and notes {notes}")
  return {"recorded":"ok"}

In [ ]:
def record_unkown_question(question):
  push(f"Recording {question} asked that I couldn't answer")
  return {"recorded":"ok"}

In [ ]:
record_user_details_json={
    "name":"record_user_details",
    "description":"Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters":{
        "type":"object",
        "properties":{
            "email":{
                "type":"string",
                "description":"the email address of user"
            },
            "name":{
                "type":"string",
                "description":"name of the user"
            },
            "notes":{
                "type":"string",
                "description":"any additional information about the conversation that is worth recording to give context"
            }
        },
        "required":["email"],
        "additionalProperties":False
    }
}

In [ ]:
record_unkown_question_json={
    "name":"record_unkown_question",
    "description":"Always use this tool to record any question that couldn't be answered as you didn't know the anwser",
    "parameters":{
        "type":"object",
        "properties":{
            "question":{
                "type":"string",
                "description":"The question that couldn't be answered"
            }
        },
        "required":["question"],
        "additionalProperties":False
    }
}

In [ ]:
tools=[{"type":"function","function":record_user_details_json},
       {"type":"function","function":record_unkown_question_json}]

In [ ]:
from ast import arguments
def handle_tool_calls(tool_calls):
  result=[]
  for tool_call in tool_calls:
    tool_name=tool_call.function.name
    arguments=json.loads(tool_call.function.arguments)
    print(f"tool called: {tool_name}",flush=True)

    if tool_name=="record_user_details":
      result=record_user_details(**arguments)
    elif tool_name=="record_unkown_question":
      result=record_unkown_question(**arguments)

    result.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool_call.id})

  return result


In [ ]:
globals()["record_unkown_question"]("This is a really hard question")

push: Recording This is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [ ]:
# Refine handle_tool_calls function using globals instead of if-statement
from ast import arguments
def handle_tool_calls(tool_calls):
  results=[]
  for tool_call in tool_calls:
    tool_name=tool_call.function.name
    arguments=json.loads(tool_call.function.arguments)
    print(f"tool called: {tool_name}",flush=True)

    tool=globals().get(tool_name)
    result=tool(**arguments) if tool else {}

    results.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool_call.id})

  return results


# Getting Context Data

In [ ]:
loader=PdfReader("/content/Profile (2).pdf")
linkedin=''
for page in loader.pages:
  text=page.extract_text()
  if text:
    linkedin +=text

with open("/content/summary.txt","r",encoding="utf-8") as f:
  summary=f.read()

name="M Asif"

In [ ]:
system_prompt=f"You are acting as {name}.You are answering questions on {name}`s website, \
particularly questions related to {name}`s career, background, skills,education and experience. \
your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given summary of {name}`s background and LinkedIn profile which you can use to answer the questions. \
Be professional and engaging, as if talking to a potential client or future employer who came accross the website. \
If you do not know the answer to any question, use your record_unkown_question tool to record the question that you couldn`t answer \
If the user is engaging in discussion,try to steer them towards getting in touch via email: ask for thier email and record it using your record_user_details tool."

system_prompt +=f"\n\n## Summary: {summary}\n\n LinkedIn Profile: {linkedin}\n\n"
system_prompt +=f"with this context, please chat with user, always staying in character as {name}"


In [ ]:
groq_model="openai/gpt-oss-20b"

In [ ]:
def chat(message, histoy):
  messages=[{"role":"system","content":system_prompt}]
  if histoy:
    for turn in histoy:
      if len(turn)==2:
        human,assistant=turn
        messages.append({"role":"user","content":human})
        messages.append({"role":"assistant","content":assistant})
  messages.append({"role":"user","content":message})

  done=False
  while not done:
    response=groq.chat.completions.create(model=groq_model, messages=messages, tools=tools)
    finish_reason=response.choices[0].finish_reason

    if finish_reason=="tool_calls":
      message=response.choices[0].message
      tool_calls=message.tool_calls
      result=handle_tool_calls(tool_calls=tool_calls)

      messages.append(message)
      messages.extend(result)
    else:
      done=True

  return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch(share=False, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

tool called: record_user_details
push: Recording  with email  and notes User asked about education and experience
tool called: record_unkown_question
push: Recording what type musician you are? asked that I couldn't answer
tool called: record_user_details
push: Recording  with email as2132@gmail.com and notes User from Lahore.
Keyboard interruption in main thread... closing server.
